## MODULE LOADING

In [1]:
## Standard Module Loading
import os
import time
import torch
import torch.nn as nn
import sys
## Argument Variable Creation
pwd=os.getcwd()
from models import *
## Custom Module Loading
from utils import Trainer, Tester
from utils import init_device, init_model,FakeLR, WarmUpCosineAnnealingLR
import matplotlib.pyplot as plt
import scipy.io as sio
from torch.utils.data import Dataset,TensorDataset
from torch.utils.data import DataLoader

## User Configuration arguments

In [2]:
## DEFINE COMMAND LINE INTERFACE VARIABLE
sys.argv = ['CSINET_IITK_ABHISHEK.py', 
            '--gpu','0',
            '--epochs','1000',
            '--cr','4']

import argparse
parser = argparse.ArgumentParser(description='CRNet PyTorch Training')


# ========================== Indispensable arguments ==========================

#parser.add_argument('--data-dir', type=str, required=True,
 #                   help='the path of dataset.')
#parser.add_argument('--scenario', type=str, required=True, choices=["in", "out"],
 #                   help="the channel scenario")
#parser.add_argument('-b', '--batch-size', type=int, default=200, required=True, metavar='N',
#                   help='mini-batch size')
#parser.add_argument('-j', '--workers', type=int, default=10, metavar='N', required=True,
#                   help='number of data loading workers')


# ============================= Optical arguments =============================

# Working mode arguments
parser.add_argument('-e', '--evaluate', dest='evaluate', action='store_true',
                    help='evaluate model on validation set')
parser.add_argument('--pretrained', type=str, default=None,
                    help='using locally pre-trained model. The path of pre-trained model should be given')
parser.add_argument('--resume', type=str, metavar='PATH', default=None,
                    help='path to latest checkpoint (default: none)')
parser.add_argument('--seed', default=None, type=int,
                    help='seed for initializing training. ')
parser.add_argument('--gpu', default=0, type=int,
                    help='GPU id to use.')
parser.add_argument('--cpu', action='store_true',
                    help='disable GPU training (default: False)')
parser.add_argument('--cpu-affinity', default=None, type=str,
                    help='CPU affinity, like "0xffff"')

# Other arguments
parser.add_argument('--epochs', type=int, metavar='N',default=1000,
                    help='number of total epochs to run')
parser.add_argument('--cr', metavar='N', type=int, default=512,
                    help='compression ratio')
parser.add_argument('--scheduler', type=str, default='const', choices=['const', 'cosine'],
                   help='learning rate scheduler')

args = parser.parse_args()


In [3]:
device, pin_memory = init_device(args.seed, args.cpu, args.gpu, args.cpu_affinity)
batch_size=200
learning_rate = 0.005
channel=2
Nt=32
Nc=32
M = 2048//args.cr
model_dir = pwd
data_type = "SB_H_INDOOR_MIXED_OFFICE_CELL_DUAL_DOWN_PP11"  #"#"COST2100_IN"#"UMa_V_freq_subband"
reduction=args.cr
weight_decay_value=0

## Raw Data Loading(Input to Model)

In [4]:
import scipy.io as sio
import h5py as h5py
# dataset_directory =f"/home/abhishek/OneDrive/Programming_IITK/dataset/38.901_abhishek/"

# # # loading mat files from specified folder
# train_mat = sio.loadmat(os.path.join(dataset_directory, f"{train_data_type}_train.mat"))['V']
# val_mat = sio.loadmat(os.path.join(dataset_directory, f"{val_data_type}_test.mat"))['V']
# test_mat = sio.loadmat(os.path.join(dataset_directory, f"{test_data_type}_test.mat"))['V']
# #Training Dataset Loading
# data_train = torch.tensor(train_mat, dtype=torch.float32)

# # Validation data loading
# data_val = torch.tensor(val_mat, dtype=torch.float32)

# # Test data loading
# data_test = torch.tensor(test_mat, dtype=torch.float32)


dataset_directory =f"/home/abhishek/dataset/38.901_abhishek/new_dual/15MHZ_bwgap/INDOOR_CELL_DUAL/"
train_mat = sio.loadmat(os.path.join(dataset_directory, f"{data_type}_train.mat"))['x']
val_mat = sio.loadmat(os.path.join(dataset_directory, f"{data_type}_test.mat"))['x']
test_mat = sio.loadmat(os.path.join(dataset_directory, f"{data_type}_test.mat"))['x']
# #Training Dataset Loading
# data_train = torch.tensor(train_mat, dtype=torch.float32)

# # Validation data loading
# data_val = torch.tensor(val_mat, dtype=torch.float32)

# # Test data loading
# data_test = torch.tensor(test_mat, dtype=torch.float32)

#Training Dataset Loading
data_train = torch.tensor(train_mat, dtype=torch.float32).view(train_mat.shape[0], channel, Nc, Nt)

# Validation data loading
data_val = torch.tensor(val_mat, dtype=torch.float32).view(val_mat.shape[0], channel, Nc, Nt)

# Test data loading
data_test = torch.tensor(test_mat, dtype=torch.float32).view(test_mat.shape[0],channel, Nc, Nt) 

 


In [5]:
# import scipy.io as sio
# #dataset_directory ="/home/abhishek/dataset/38.901/"
# # loading mat files from specified folder
# #train_mat = sio.loadmat(os.path.join(dataset_directory, "uma_fr1_train.mat"))['x']
# #test_mat = sio.loadmat(os.path.join(dataset_directory, "uma_fr1_test.mat"))['x']
# #val_mat = sio.loadmat(os.path.join(dataset_directory, "uma_fr1_test.mat"))['x']

# #Training Dataset Loading
# data_train = torch.tensor(train_mat, dtype=torch.float32) #.view(data_train.shape[0], channel, nt, nc)

# # Validation data loading
# data_val = torch.tensor(val_mat, dtype=torch.float32) #.view(data_val.shape[0], channel, nt, nc)

# # Test data loading
# data_test = torch.tensor(test_mat, dtype=torch.float32) #.view(data_test.shape[0], channel, nt, nc) 

## Preprocessing

In [6]:
def preprocess_raw(raw_input):
    #out = raw_input.view(raw_input.shape[0],channel,Nt,Nc)
    #[samp,h,w,d]=np.shape(raw_input)
    out = raw_input
    # out = torch.zeros(raw_input.shape[0],channel,Nt,Nc)
    # for i in range(raw_input.shape[0]):
    #     x=raw_input[i,:,:,:]
    #     out[i,:,:,:] = 0.5*(x/torch.max(torch.abs(x))+1) #- 0.1*torch.rand(samp,h,w,d) 
    return out

## Preprocess & Create Tensor Dataset, Dataloader

In [7]:
from torch.utils.data import TensorDataset
import numpy as np


# Preprocessing Raw Data
data_train_pre = preprocess_raw(data_train)
data_val_pre = preprocess_raw(data_val)
data_test_pre = preprocess_raw(data_test)

# Covert Data to TensorDataset
train_tensordataset = TensorDataset(data_train_pre)
val_tensordataset = TensorDataset(data_val_pre)
test_tensordataset = TensorDataset(data_test_pre)

## Create Dataloader
from torch.utils.data import DataLoader
train_loader = DataLoader(train_tensordataset, batch_size = batch_size, num_workers=16,shuffle = True)
test_loader = DataLoader(test_tensordataset, batch_size = batch_size,num_workers=16, shuffle = False)
val_loader = DataLoader(val_tensordataset, batch_size = batch_size,num_workers=16, shuffle = False)

## Model Define

## MODEL Initialization and Loading into Device

In [8]:
#CsiNet = CsiNet_module(in_channels, input_height, input_width, M)
train_model = CSINET_ANN(M)
train_model = train_model.to(device)

## Define Optimizer and Criterion

In [9]:
optimizer = torch.optim.Adam(params = train_model.parameters(), lr = learning_rate, weight_decay=weight_decay_value)
criterion_mse = nn.MSELoss(reduction='mean').to(device)

In [10]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.005
    maximize: False
    weight_decay: 0
)

In [11]:
if args.scheduler == 'const':
 scheduler = FakeLR(optimizer=optimizer)

else:
 scheduler = WarmUpCosineAnnealingLR(optimizer=optimizer,
                                 T_max=args.epochs * len(train_loader),
                                 T_warmup=30 * len(train_loader),
                                 eta_min=5e-5)

## Train Model

In [12]:
# training function
# CsiNet.load_state_dict(torch.load(model_weights_file))
trainer = Trainer(model=train_model,
           device=device,
           optimizer=optimizer,
           criterion=criterion_mse,
           scheduler=scheduler,
           resume=args.resume)
trainer.loop(args.epochs, train_loader, val_loader, test_loader)


I 12.23/20:08 solver.py:91  ] Epoch: [1/1000][100/240] lr: 5.00e-03 | MSE loss: 1.69211e-02 | time: 0.385
I 12.23/20:08 solver.py:91  ] Epoch: [1/1000][200/240] lr: 5.00e-03 | MSE loss: 1.03164e-02 | time: 0.226
I 12.23/20:08 solver.py:91  ] => Train  Loss: 8.95409e-03

I 12.23/20:08 solver.py:91  ] Epoch: [2/1000][100/240] lr: 5.00e-03 | MSE loss: 1.60715e-03 | time: 0.071
I 12.23/20:08 solver.py:91  ] Epoch: [2/1000][200/240] lr: 5.00e-03 | MSE loss: 1.40363e-03 | time: 0.071
I 12.23/20:08 solver.py:91  ] => Train  Loss: 1.34875e-03

I 12.23/20:08 solver.py:246 ] [100/120] MSE loss: 1.05258e-03 | time: 0.030

=> Test result: 
loss: 1.05333e-03    rho: 9.981e-01    NMSE: -1.36789e+01

I 12.23/20:08 solver.py:246 ] [100/120] MSE loss: 1.05258e-03 | time: 0.028

=> Test result: 
loss: 1.05333e-03    rho: 9.981e-01    NMSE: -1.36789e+01


=! Best rho: 9.981e-01 (Corresponding nmse=-1.368e+01; epoch=2)
   Best NMSE: -1.368e+01 (Corresponding rho=9.981e-01;  epoch=2)

I 12.23/20:08 solver.

In [13]:
args.epochs=60
trainer.loop(args.epochs, train_loader, val_loader, test_loader)

## Save Trained Model

In [14]:
model_dir_train = f"{pwd}/trained_models"
#Save model
torch.save(train_model.state_dict(), f'{model_dir_train}/{data_type}_lerning={learning_rate}_reduction={reduction}_weight_decay={weight_decay_value}.pt')
dummy = torch.randn(1,channel,Nc,Nt).to(device)
torch.onnx.export(train_model,dummy,f'{model_dir_train}/{data_type}_lerning={learning_rate}_reduction={reduction}_weight_decay={weight_decay_value}.onnx')

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



## Load Trained Model for Testing

In [15]:
# model_dir_test ="/home/abhishek/OneDrive/Programming_IITK/summer_interns_iitk_2023/Bayesian NN IITK 3rd Year/Abhishek/trained_models/CSINET_ANN"
# use_pre_train = True
# test_model_weight_decay =0
# test_model_datatype="UMi_V_freq_subband"
# test_model_name = f"{test_model_datatype}_reduction={reduction}_weight_decay={test_model_weight_decay}"
# model_file=f'{model_dir_test}/{test_model_name}.pt'
# test_model = csinet2(args.cr)
# test_model = test_model.to(device)
# if use_pre_train:
#    # Load Pretrained Saved Model
#     test_model.load_state_dict(torch.load(model_file)) 

## Testing Function

In [19]:
import torch
import numpy as np

# 1.NMSE FUNCTION
def NMSE(Hi, Hi_hat):
	Hi_real = torch.reshape(Hi[:, 0, :, :], (Hi.size(0), -1))
	Hi_imag = torch.reshape(Hi[:, 1, :, :], (Hi.size(0), -1))
	Hi_comp = (Hi_real-0.5) + 1j * (Hi_imag-0.5)

	Hi_hat_real = torch.reshape(Hi_hat[:, 0, :, :], (Hi_hat.size(0), -1))
	Hi_hat_imag = torch.reshape(Hi_hat[:, 1, :, :], (Hi_hat.size(0), -1))
	Hi_hat_comp = (Hi_hat_real-0.5) + 1j * (Hi_hat_imag-0.5)

	mse = torch.sum(torch.square(abs(Hi_comp - Hi_hat_comp)), axis = 1)
	power = torch.sum(torch.square(abs(Hi_comp)), axis = 1)
	# print((power))

	nmse = (torch.mean(mse / power))
# 10*torch.log10(torch.mean(mse)).item(),
	return nmse.item()

# 2.validation function
def test_avg(model,val_loader,device,num_test_networks):
    print('testing')
    
    # test_input_dataloader=torch.utils.data.DataLoader(test_input, batch_size=B, shuffle=True)
    output_tensor=0
    for i in range(num_test_networks):
# Create an empty list to store the output tensors
        output_tensors = []
        input_tensor = []
        model.eval()  # Set the model in evaluation mode
    
        with torch.no_grad():
            for batch_idx,(inputs,) in enumerate(val_loader):
                inputs=inputs.to(device)
                outputs = model(inputs)
                output_tensors.append(outputs)
                input_tensor.append(inputs)
    

# Concatenate the output tensors along the batch dimension
        output_tensor = output_tensor+torch.cat(output_tensors, dim=0)
    input_tensor=torch.cat(input_tensor, dim=0)
    return NMSE(input_tensor.to(device),output_tensor/num_test_networks)


def test_instant(model,val_loader,device):
    print('testing')
    
    # test_input_dataloader=torch.utils.data.DataLoader(test_input, batch_size=B, shuffle=True)

    
# Create an empty list to store the output tensors
    nmse_out=[]
    model.eval()  # Set the model in evaluation mode
    
    with torch.no_grad():
        for batch_idx,(inputs,) in enumerate(val_loader):
            inputs=inputs.to(device)
            outputs = model(inputs)
            nmse_out.append(NMSE(inputs,outputs))

# Concatenate the output tensors along the batch dimension
    return nmse_out
    

## Testing Model and Final KPI

In [17]:
num_test_networks=1
import scipy.io as sio
criterion_mse = nn.MSELoss().to(device)
# dataset_directory ="/home/abhishek/OneDrive/Programming_IITK/dataset/38.901/"
# test_dataset2 = "SB_H_UMI_NLOS_UE1_PP1"
# test_mat2 = sio.loadmat(os.path.join(dataset_directory, f"{test_dataset2}_test.mat"))['x']

# # Test data loading
# data_test2 = torch.tensor(test_mat2, dtype=torch.float32)#.view(test_mat2.shape[0],channel, Nt, Nc) 

# from torch.utils.data import Dataset,TensorDataset
# # Preprocessing Raw Data
# data_test_pre2 = preprocess_raw(data_test2)

# # Covert Data to TensorDataset
# test_tensordataset2 = TensorDataset(data_test_pre2)

# ## Create Dataloader
# from torch.utils.data import DataLoader
test_loader2 = DataLoader(test_tensordataset, batch_size = 100,num_workers=16,pin_memory=pin_memory, shuffle = False)
# test_loader4 = DataLoader(test_tensordataset, batch_size = 100,num_workers=16,pin_memory=pin_memory, shuffle = False)
# # Final testing
# test_model=train_model
# loss, rho, nmse = Tester(test_model, device, criterion_mse)(test_loader2)
# print(f"\n=! Final test loss: {loss:.3e}"
#    f"\n         test rho: {rho:.3e}"
#    f"\n         test NMSE: {nmse:.3e}\n")

for i in range(num_test_networks):
    model1 = train_model
    nmse_out = test_instant(model1,test_loader2,device)
    # nmse_out2 = test_instant(model1,test_loader4,device)

# print(nmse_out)
# print(f"avg = {np.mean(nmse_out): .2e} \n max = {np.max(nmse_out):.2e} \n min = {np.min(nmse_out):.2e}")
print(f"avg = {(10*np.log10(np.mean(nmse_out))): .2e} \n worst= {10*np.log10(np.max(nmse_out)):.2e} \n best = {10*np.log10(np.min(nmse_out)):.2e}")
# import matplotlib.pyplot as plt
# title_name=f'ANN_NMSE_decay={weight_decay_value}_lr={learning_rate},test={test_dataset},train={train_data_type}'
# plt.title(title_name)
# plt.hist(nmse_out)
# plt.xlabel('NMSE')
# # plt.xlim((-50,20))
# plt.ylabel('UE DESNITY')
# plt.savefig(fname=f'{pwd}/results/{title_name}',format='jpeg')

testing
avg = -2.17e+01 
 worst= -2.07e+01 
 best = -2.20e+01


## Analyse Dataset

In [18]:
title_name=f'ANN_NMSE_decay={weight_decay_value}_lr={learning_rate},test={test_dataset2},train={train_data_type}'
plt.title(title_name)
plt.hist(nmse_out)
plt.xlabel('NMSE')
plt.xlim((-50,20))
plt.ylabel('UE DESNITY')

NameError: name 'test_dataset2' is not defined

In [ ]:
plt.hist(train_mat)

In [ ]:
x=data_train.reshape(data_train.shape[0],-1)
plt.hist(x)

In [ ]:
10.1/10

In [ ]:
x=[1,2]
z= [10**(y//10) for y in x]
